In [ ]:
import os
import pandas as pd
import io
import unicodedata

root_dir = 'coral_and_sponges_all/data/pub/data/paleo/coral'


In [ ]:
root_files = os.listdir(root_dir)
root_files = root_files[0:3] + root_files[4:6] + root_files[9:12]
sub_files_to_flatten = [[f'{r}/{f}' for f in os.listdir(f'{root_dir}/{r}')] for r in root_files]
#Flatten list of files into one list
sub_files = [f'{root_dir}/{item}' for sublist in sub_files_to_flatten for item in sublist]

In [ ]:
len(sub_files)

In [89]:
def create_coral_file_df(filename):
    # print("reading")
    try:
        # Using a weird encoding to solve file read issue; see https://stackoverflow.com/questions/46000191/utf-8-codec-cant-decode-byte-0x92-in-position-18-invalid-start-byte
        coral_file = open(filename,'r', encoding="ascii", errors="ignore")
        file_in = coral_file.readlines()
        if file_in[0][0] != "#":
            print(f"'{filename}' does not begin with '#'; COMPLETELY IGNORING IT")
            return None

        for i in range(len(file_in)):
            line = file_in[i]
            if "Location:" in line:
                location = line.split(": ")[1].strip()
            if "Northernmost_Latitude:" in line:
                northernmost_lat = line.split(": ")[1].strip()
            if "Southernmost_Latitude:" in line:
                southernmost_lat = line.split(": ")[1].strip()
            if "Easternmost_Longitude:" in line:
                easternmost_lat = line.split(": ")[1].strip()
            if "Westernmost_Longitude:" in line:
                westernmost_lat = line.split(": ")[1].strip()
            if "Elevation:" in line:
                elevation = line.split(": ")[1].split(' ')[0].strip()
            if "Earliest_Year:" in line:
                earliest_year = line.split(": ")[1].strip()
            if "Most_Recent_Year:" in line:
                most_recent_year = line.split(": ")[1].strip()
            if "Species_Name:" in line:
                species = line.split(": ")[1].strip('\n').strip()
            if not line.startswith('#'):
                if not line[0].isprintable():
                    continue # Don't break on weird, non-printable characters
                elif len(line.strip()) == 0:
                    continue
                else:
                    age_line = i
                    extracted = ' '.join(file_in[age_line:])
                    break #exit the for loop, we have obtained the data!

        lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
        lon = (float(easternmost_lat) + float(westernmost_lat)) / 2

        data = io.StringIO(extracted)
        df = pd.read_csv(data, sep="\t", error_bad_lines=False)

        df.columns = (map(lambda x: x.lower(), df.columns))

        # find the correct 'age' column
        if 'age' not in df.columns:
            if 'age_ad' in df.columns:
                df['age'] = df['age_ad']
            elif 'age_calad' in df.columns:
                df['age'] = df['age_ad']
            elif 'time' in df.columns:
                df['age'] = df['time']
            elif 'date' in df.columns:
                df['age'] = df['date']
            elif 'year' in df.columns:
                df['age'] = df['year']
            elif 'year_ce' in df.columns:
                df['age'] = df['year_ce']
            else:
                print("No 'age' column header detected.")

        #print(df['age'])

        #find the correct d18O column
        if 'd18o' not in df.columns:
            count = 0
            for col in df.columns:
                # we are assuming that the first col with d18O is the one we want
                if 'd18o' in col:
                    if 'recon' in col:
                        continue
                    df['d18o'] = df[col]
                    break
        
        #print(df['d18o'])

        df = df[['age', 'd18o']]
        df = df[df['d18o'] != 'NAN']

        # average the d18O values across the years
        df['year']  = df['age'].apply(lambda x: str(x).strip()[0:4])
        df = df.groupby(by="year")["d18o"].mean().reset_index('year')

        df['lat'] = lat
        df['lon'] = lon
        df['elevation'] = elevation
        
        try:
            species
            df['species'] = species
        except NameError:
            df['species'] = 'Other'

        return(df)

        # except Exception as e:
        #     print("exception2")
        #     print("ERROR : "+str(e))
        #     # print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
            # print('hi')
    except Exception as e:
        print(filename)
        print("ERROR : "+str(e))

Quick Proof of Concept for the function above:

In [92]:
#filename = f'{root_dir}/atlantic/barbados_coral18o.txt'
# filename = f'{root_dir}/atlantic/goodkin2008-sst_noaa.txt'
# filename = f'{root_dir}/atlantic/smith2006-lk1.txt'
# filename = f'{root_dir}/red_sea/felis2000_noaa.txt'
# filename = f'{root_dir}/atlantic/moses2006-side_noaa.txt'
# filename = f'{root_dir}/atlantic/barbados_coral18o.txt'
filename = f'{root_dir}/west_pacific/gorman2012.txt'
#'swart1998-st-1_noaa.txt'

create_coral_file_df(filename)

coral_and_sponges_all/data/pub/data/paleo/coral/west_pacific/gorman2012.txt
ERROR : local variable 'northernmost_lat' referenced before assignment


In [91]:
all_coral_df = pd.DataFrame(columns=['age', 'd18O', 'lat', 'lon', 'elevation', 'species'])
for file in sub_files:
    # print(file)
    coral_df = create_coral_file_df(file)
    try:
        all_coral_df = pd.concat([all_coral_df, coral_df])
    except Exception as e:
        print('Error: ' + str(e))

del all_coral_df['age']
del all_coral_df['d18O']

all_coral_df.reset_index
all_coral_df


ipykernel_launcher:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


b'Skipping line 219: expected 5 fields, saw 6\n'


coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/kuhnert2002_noaa.txt
ERROR : Could not convert -3.99-4-4.11-4.16-4.35-4.13-4.07-4.15-3.88-3.81-3.37-3.25-3.26-3.23-3.15 to numeric
'coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/barbados_coral18o.txt'' does not begin with '#'; COMPLETELY IGNORING IT
'coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/kuhnert2005-fixed.txt'' does not begin with '#'; COMPLETELY IGNORING IT
No 'age' column header detected.
coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/kuhnert2002-inter_noaa.txt
ERROR : "['age'] not in index"
'coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/bahamas-sclerosponge2005.txt'' does not begin with '#'; COMPLETELY IGNORING IT
'coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/bahamas-sclerosponge-2002.txt'' does not begin with '#'; COMPLETELY IGNORING IT
coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/kuhnert2005
ERROR : [Errno 21] Is a directory: 'coral_and_sponges_all/da

b'Skipping line 1419: expected 3 fields, saw 4\n'


No 'age' column header detected.
coral_and_sponges_all/data/pub/data/paleo/coral/west_pacific/brenner2017-her.txt
ERROR : "['age'] not in index"
'coral_and_sponges_all/data/pub/data/paleo/coral/west_pacific/palau2002.txt'' does not begin with '#'; COMPLETELY IGNORING IT
'coral_and_sponges_all/data/pub/data/paleo/coral/west_pacific/okinawa2006.txt'' does not begin with '#'; COMPLETELY IGNORING IT
'coral_and_sponges_all/data/pub/data/paleo/coral/west_pacific/kikai2004.txt'' does not begin with '#'; COMPLETELY IGNORING IT
'coral_and_sponges_all/data/pub/data/paleo/coral/west_pacific/guam2004.txt'' does not begin with '#'; COMPLETELY IGNORING IT
No 'age' column header detected.
coral_and_sponges_all/data/pub/data/paleo/coral/west_pacific/Murty.MakassarStrait.2017-1.txt
ERROR : "['age'] not in index"
coral_and_sponges_all/data/pub/data/paleo/coral/west_pacific/cole1993-ldgo-39_noaa.txt
ERROR : Could not convert -5.05-5.056-4.743-4.697-4.844-5.24-5.062-4.826-4.899-4.812-5.168-4.723 to numeri

,lat,lon,elevation,species,year,d18o
0,11.17,-60.85,-4,Montastaea faveolata,1927,-4.067500
1,11.17,-60.85,-4,Montastaea faveolata,1928,-3.865000
2,11.17,-60.85,-4,Montastaea faveolata,1929,-3.865000
3,11.17,-60.85,-4,Montastaea faveolata,1930,-4.007500
4,11.17,-60.85,-4,Montastaea faveolata,1931,-4.040000
...,...,...,...,...,...,...
2,9.83,115.50,-5,Other,1998,-5.974038
3,9.83,115.50,-5,Other,1999,-5.802692
4,9.83,115.50,-5,Other,2000,-5.867736
5,9.83,115.50,-5,Other,2001,-5.959808


In [ ]:
all_coral_df.to_csv('all_coral.csv')


In [ ]:
all_coral_df

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(all_coral_df['lon'], all_coral_df['lat'])


In [ ]:
import shapely
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

geometry = [Point(xy) for xy in zip(all_coral_df['lon'], 
                                    all_coral_df['lat'])]

gdf = GeoDataFrame(all_coral_df, geometry=geometry)

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

Looking at the lat/lon of our data, it looks like we will have a model heavily biased towards the western part of this map, and in tropical regions.

In [ ]:
coral_file = 'swart1998-st-1_noaa.txt'
# Example of file with -noaa extension
file1 = open(coral_file,'r')
print(file1.read())
file1.close()

Gather metadata info! (site lat/lon & years of study)

In [ ]:
file1 = open(coral_file,'r')
file_in = file1.readlines()

for line in file_in:
    if "Location:" in line:
        location = line.split(": ")[1]
    if "Northernmost_Latitude:" in line:
        northernmost_lat = line.split(": ")[1]
    if "Southernmost_Latitude:" in line:
        southernmost_lat = line.split(": ")[1]
    if "Easternmost_Longitude:" in line:
        easternmost_lat = line.split(": ")[1]
    if "Westernmost_Longitude:" in line:
        westernmost_lat = line.split(": ")[1]
    if "Elevation:" in line:
        elevation = line.split(": ")[1].split(' ')[0]
    if "Earliest_Year:" in line:
        earliest_year = line.split(": ")[1]
    if "Most_Recent_Year:" in line:
        most_recent_year = line.split(": ")[1]
    if "Species_Name:" in line:
        species = line.split(": ")[1].strip('\n')

lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
lon = (float(easternmost_lat) + float(westernmost_lat)) / 2



In [ ]:
age_line = 0
file1 = open(coral_file,'r')
with file1 as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('age'):
            print(i)
            age_line = i
            break
    extracted = ' '.join(lines[age_line:])
    print(extracted)

In [ ]:
data = io.StringIO(extracted)
df_test = pd.read_csv(data, sep="\t")
df_test.head()

Just grab `age` and `d18O` columns! And remove NAN values. 

In [ ]:
df_test_d18O = df_test[['age', 'd18O']]
df_test_d18O = df_test_d18O[df_test_d18O['d18O'] != 'NAN']
df_test_d18O['lat'] = lat
df_test_d18O['lon'] = lon
df_test_d18O['elevation'] = elevation
df_test_d18O['species'] = species
df_test_d18O